In [5]:
import pandas as pd
d = {"headers":{"Candidate":["id","name"],"Vote":["id","candidateId"]},"rows":{"Candidate":[[1,"A"],[2,"B"],[3,"C"],[4,"D"],[5,"E"]],"Vote":[[1,2],[2,4],[3,3],[4,2],[5,5]]}}
pd.DataFrame(d['rows']['Candidate'], columns=d['headers']['Candidate']).to_csv("./Candidate.txt", index=None)
pd.DataFrame(d['rows']['Vote'], columns=d['headers']['Vote']).to_csv("./Vote.txt", index=None)

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("574").config("pyspark.sql.shuffle.partition","4").getOrCreate()

In [6]:
c = spark.read.option("header", True).csv("./Candidate.txt")
v = spark.read.option("header", True).csv("./Vote.txt")
# e = e.withColumn("id", e.id.cast('int')).withColumn("managerId", e.managerId.cast('int'))
c.show()
v.show()

+---+----+
| id|name|
+---+----+
|  1|   A|
|  2|   B|
|  3|   C|
|  4|   D|
|  5|   E|
+---+----+

+---+-----------+
| id|candidateId|
+---+-----------+
|  1|          2|
|  2|          4|
|  3|          3|
|  4|          2|
|  5|          5|
+---+-----------+



Write an SQL query to report the name of the winning candidate (i.e., the candidate who got the largest number of votes).

The test cases are generated so that exactly one candidate wins the elections.

The query result format is in the following example.

In [7]:
votes = v.groupBy("candidateId").count()
c.join(votes, c.id==votes.candidateId, "leftouter").orderBy("count", ascending=False).select("name").first()

Row(name='B')